In [1]:
from IPython.display import HTML

HTML(
    """<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>."""
)

In [2]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))
display(HTML("<style>.output_result { max-width:90% !important; }</style>"))

In [3]:
%%html
<style>
div.jupyter-widgets.widget-label {display: none;}
</style>

In [4]:
import numpy as np
import sys
import os
import re
import matplotlib.pyplot as plt
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
import matplotlib

matplotlib.rcParams["figure.facecolor"] = "w"
matplotlib.rcParams["figure.autolayout"] = True
matplotlib.rcParams["figure.figsize"] = (10, 5)

from scipy.stats import binom, beta, multivariate_normal, uniform
import scipy.integrate as integrate

import ipywidgets as widgets
from ipywidgets import interact


%matplotlib widget

# Bayesian inferance 
## Combining MCMC and surrogate models

### Stéphane Nilsson


## Bayes' theorem

Let's first give the mathematical description of Baye's theorem, and then we will go through the parameters and have a concrete exemple.

* $P(\theta | y)= \frac{P(y|\theta)P(\theta)}{P(y)}$
  * $y$: Observed data
  * $\theta$: Parameters


* $P(\theta)  :$ Prior probability density
* $P(y|\theta):$ Likelihood
* $P(y)       :$ Model evidence
* $P(\theta|y):$ Posterior probability density -> What we are interested in

The prior probability density, which I will reduce to prior probability, is the probability of our parameters before taking data into account and quantifies the uncertainty about the parameters. It could be a Uniform distribution which bounds the parameter space to values we know it cannot exceeed. For exemple, if we had to make a linear fit with positive slope, we could bound the values to be positive by setting a Uniform distribution in [0, 1e5].

The likelihood tells you how likely the data you have could have been generated by the parameter. Continuing on the linear regression, let's suppose the real data is coming from y=3x. Then the likelihood function would be greated if my parameter of the problem, the slope, is 2.5 rather than 0.5, since on average each point would be closer to my y=2.5x than to my y=0.5x line. The likelihood function is problem dependant as we will see in the following exemples.

The model evidence is the probability of having the data given the model. Known also as the marginal likelihood, it is a constant given a single model. This will give our normalizing constant to have a proper probability distribution.

The posterior probability is the probability that our parameters have effectively generated the data. In the case of bayseian inferance, we have the data and would like to obtain the underlying parameters, therefore this is the value we eventually want to compute.


## Linear regression exemple

Let's start by creating a toy example : $$y=4x+1+\epsilon$$

where we set $\epsilon \sim \mathcal{N}(0, 0.5)$

In that case we can define our parameter vector $\theta = \begin{bmatrix} a \\ b \end{bmatrix} $ such that $\mathbf{y}^*=\theta^T\cdot\mathbf{x}$ are our estimated values.

We would therefore estimate the best values of a and b that fit our data.

In [5]:
np.random.seed(0)
x=np.linspace(0,1)
y=4*x+1
x_train=np.random.random(10)
y_train=4*x_train+1+np.random.normal(0,0.5,10)
fig, ax=plt.subplots(1,1)


ax.plot(x_train, y_train, '.', label='Training data points', color='purple')
ax.plot(x,y, label='True function', color='orange')
ax.legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Let's set up our problem specifications :

* We want to evaluate the parameter $\theta$ of our linear regression. We remind that it is composed of the slope and the intercept, $a$ and $b$.
* Looking at the data point, we already know that we have a positive slope. Other than that, we can only estimate a range for the values. We propose a Uniform distribution as a prior for both and specify $P(a)$=Uniform(0,10) and $P(b)=$Uniform(-5,5).
* The likelihood is given by $P(y|\theta)=\frac{1}{(\sqrt{2\pi}\sigma)^n}\exp\left(-\frac{1}{2\sigma^2}(\mathbf{y}- \mathbf{y}^*)^{\rm T}(\mathbf{y}- \mathbf{y}^*)\right)$ with $n$ the number of data points. (10 in this example)
* Finally, we get the posterior $P(\theta|\mathbf{y})\sim P(y|\theta)P(a)P(b)$

For numerical stability, we can instead compute the log-likelihood $$\log(P(\theta|\mathbf{y})= \log(P(y|\theta))+\log(P(a))+\log(P(b))$$
With $\log(P(a))$ and $\log(P(b))$ being constant, we can remove them from the equation. We thus get $$\log(P(\theta|\mathbf{y}))\sim -\frac{1}{2\sigma^2}(\mathbf{y}-\mathbf{y^*})^2$$

The best parameters would therefore minimize the log-likelihood. One the plot below, a colormap of the likelihood as a function of the two parameters can be created. The best parameters will be in dark purple. A contour map can also be plotted to give an idea of how the likelihood function is varying.

In [6]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
x=np.linspace(0,1,2)
y=4*x+1

cmap = matplotlib.cm.get_cmap("plasma")

cmap.set_bad(color='white')


flag=np.ones((21,21))
flag_colormap=1
output = widgets.Output()

def posterior_(a, b, y):
    y = prior_(a,b) * likelihood_(y)
    return y 

def log_posterior_(y):
    y = log_likelihood(y)
    return y 

def likelihood_(y):
    y = multivariate_normal.pdf(y,mean=y_train,cov=0.5)
    return y

def log_likelihood(y):
    y = -multivariate_normal.logpdf(y,mean=y_train,cov=0.5)
    return y

# Just showing how we can compute the uniform probability with scipy
def prior_(a ,b ):
    return uniform.pdf(a, 0, 10)*uniform(b, -5 ,5)
###############################################################################
z_post=np.array([log_posterior_(a*x_train+b) for a in np.linspace(-10,10,21) for b in np.linspace(-10,10,21)])
z_post=z_post.reshape((21,21))
#Make the transition smoother between colors
z_post=np.log(z_post)
###############################################################################
@interact(
    a=widgets.IntSlider(min=-10, max=10, value=1, description="Slope a: "),
    b=widgets.IntSlider(min=-10, max=10, value=1, description="Intercept b: "),
)
def update(a, b):
    # [l.remove() for l in ax.lines]
    # [l.remove() for l in ax.lines]
    # Clear axis
    global flag_colormap
    axs[0].cla()
    # Check if ax.collections is empty
    # if len(ax.collections):
    #     ax.collections.pop()
    y_post= a*x+b
    flag[a+10,b+10]=0
    z_masked=np.ma.masked_where(flag, z_post)
    #z_post[a+9,b+9]=log_posterior(y_post_train)


    axs[0].plot(x_train, y_train, '.', label='Observered data', color='purple')
    axs[0].plot(x, y_post, label='Regression line', color='orange')

    axs[0].legend()

    axs[0].set_ylim(
        (np.min(np.r_[y, y_post]) - 1e-1, np.max(np.r_[y, y_post]) + 1e-1)
    )
    axs[0].set_xlim((0, 1))

    img=axs[1].imshow(z_masked,cmap=cmap, extent=[-10,10,10,-10],vmin=np.min(z_post), vmax=np.max(z_post))
    axs[1].set_xticks(ticks=np.linspace(-10,10,11))
    axs[1].set_yticks(ticks=np.linspace(-10,10,11))
    axs[1].set_xlabel('b', fontsize=16)
    axs[1].set_ylabel('a', fontsize=16)
    #Avoid plotting the colormap multiple times
    if flag_colormap:
        plt.colorbar(img,ax=axs[1],shrink=0.75);
        flag_colormap=0



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='Slope a: ', max=10, min=-10), IntSlider(value=1, descrip…

In [7]:
fig, ax=plt.subplots(1,1)
param=np.linspace(-10,10,101)
z_post2=np.array([log_posterior_(a*x_train+b) for a in param for b in param])
z_post2=z_post2.reshape((101,101))
#Make the transition smoother between colors
z_post2=np.log(z_post2)
row, col = np.unravel_index(z_post2.argmin(), z_post2.shape)
print(f'The best parameter set is a={param[row]:0.2f} and b={param[col]:0.2f}')
ax.contour(param,param,z_post2,cmap=cmap)
ax.plot(param[col],param[row],'k*', ms=8 ,label='Best parameters')
ax.legend();


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The best parameter set is a=4.00 and b=1.20


## Coin flip exemple
Context : You are at a fair and on one of the stand, a mysterious person tells you to come by. He tells you the following simple rules : he will throw the coin 10 times in the air. If you guess correctly the number of head, you double your bet, if the number of head is higher, then you lose your bet, if the number is lower, you get back your bet. However you feel like something is wrong and that the coin is biased. But to make sure, you have to gather evidence, and therefore wait to see 5 people playing before you. We will now apply Bayesian reasoning to see if your intuition was real.

In the context of a coin flip, we can compute what is the probability of obtaining N heads out of 10 throws using the Binomial distribution Binomial(N,10,p), where $p=0.5$ is the probability of getting a head. In a fair coin case, the probability $p$ is 0.5. Below is a graph showing the probability of getting N heads out of 10 throws for N=0...10. In that case the probability is highest to get 5 heads and 5 tails, which makes sense since p=0.5. Getting 8 heads has a 5% chance, but it's not impossible. Therefore, if participants are losing, is it bad luck or is there something else going on ? Let's explore this.



In [8]:
fig, ax= plt.subplots(1,1)
x=np.arange(0,11)
ax.plot(x, binom.pmf(x, 10, 0.5), 'bo', ms=5)
ax.set_ylabel('Probability')
ax.set_xlabel('Number of heads');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

First we have to define our problem. Keep in mind that we want to evaluate the parameter $p$ of the Binomial distribution, which is a probability distribution. In the end, we will have a probability distribution of the value $p$, that is we will not get the best value of $p$, but a plausible range.

* We want to evaluate the probability $p$ of getting a head ($\mathrm{H}$)
* We start by assuming we have no prior information on the coin, thus we set the prior $P(p)$=Beta(1,1) (Equivalent to Uniform distribution)
* The likelihood is given by $P(\mathrm{H}|p)$=Binomial($\mathrm{H}$, $\mathrm{H+T}$, $p$)
* Finally, we get the posterior $P(p|H)\sim $ Beta(1,1)xBinomial($\mathrm{H}$, $\mathrm{H+T}$, $p$)

For an analytical derivation of the solution, visit [Bayesian Coin Flip](https://nbviewer.org/github/lambdafu/notebook/blob/master/math/Bayesian%20Coin%20Flip.ipynb)

For more in-depth explanations with Python code, I recommand [Bayesian Methods for Hackers](https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers)

In [9]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))


def posterior(H, T, a, b, x):
    y = prior(a, b, x) * likelihood(H, T, x)
    return y / integrate.simpson(y, x)


def likelihood(H, T, x):
    # Normalize likelihood to make it look nicer
    y = binom.pmf(H, H + T, [x]).reshape(-1,)
    return y / integrate.simpson(y, x)


def prior(a, b, x):
    return beta.pdf(x, a, b)


@interact(
    H=widgets.IntSlider(
        min=0,
        max=50,
        value=0,
        description="Number of heads: ",
        style={"description_width": "initial"},
    ),
    T=widgets.IntSlider(
        min=0,
        max=50,
        value=0,
        description="Number of tails:   ",
        style={"description_width": "initial"},
    ),
    a=widgets.IntSlider(min=1, max=10, value=1, description="a: "),
    b=widgets.IntSlider(min=1, max=10, value=1, description="b: "),
)
def update(H, T, a, b):
    # [l.remove() for l in ax.lines]
    # [l.remove() for l in ax.lines]
    # Clear axis
    ax.cla()
    # Check if ax.collections is empty
    # if len(ax.collections):
    #     ax.collections.pop()
    x = np.linspace(0, 1, 200)

    y_prior = prior(a, b, x)
    y_likelihood = likelihood(H, T, x)
    y_post = posterior(H, T, a, b, x)

    ax.plot(x, y_prior, "--", c="blue", label="Prior", alpha=0.6)
    ax.plot(x, y_likelihood, "--", c="red", label="Likelihood", alpha=0.6)
    ax.plot(x, y_post, c="violet", label="Posterior")
    ax.fill_between(x, x * 0, y_post, color="violet", alpha=0.2)

    ax.set_xlabel("$p$")
    ax.set_ylabel("PDF")
    ax.legend()

    ax.set_ylim(
        (np.min(y_post) - 1e-1, np.max(np.r_[y_post, y_likelihood, y_prior]) + 1e-1)
    )
    ax.set_xlim((0, 1))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Number of heads: ', max=50, style=SliderStyle(descriptio…

If we look at the relation $P(\theta|y)\propto P(\theta)*P(y|\theta)$, the only thing missing is $P(y)$ which will act as the normalizing constant. However, computing $P(y)=\int P(y|\theta)d\theta$ requires computing an integral. Numerous methods are readily available when computing one-dimensional integrals, however, in an N-dimensional space (that is we have N parameters), the integral is computationaly too expensive, due to the curse of dimensionality. Therefore other methods can be used, such as the MCMC class of algorithms.

## What if we cannot compute directly $P(\theta|y)$?
### Markov chain Monte Carlo (MCMC)

Monte Carlo : Generation of random numbers from a given distribution

Markov Chain : Sequence of number where each number is dependant on the previous number

Metropolis-Hastings : Defining an acceptance rate of the new number

The algorithm will sample the N-dimensional space with a probability proportional to our posterior probability.

Algorithm :
1. Initialize with an arbitrary set of parameters $\theta$ and set a probability distribution function from which the next candidate $\theta'$ will be picked from. Usually it is a Gaussian centered around $\theta$.
2. Randomly pick a new candidate $\theta'$ following the distribution.
3. Compute the acceptance probability $\alpha=\frac{P(y|\theta')P(\theta')}{P(y|\theta)P(\theta)}$
4. Accept or reject :
  * Draw a random number $u \in [0,1]$
  * If $\alpha>u$ , accept and set $\theta_{t+1}=\theta'$
  * If $\alpha<u$ , reject and set $\theta_{t+1}=\theta$

Discard the first $N$ values as burn-in period to keep converged values.

For more information about different sampling techniques, visit [MCMC Algorithms](https://m-clark.github.io/docs/ld_mcmc/index_onepage.html) 

Now let's apply the algorithm to the previous example and compare to analytical solution as the number of samples grows.

This time we assume that the coin is biased towards Heads, and thus set the prior distribution to Beta(10,5).

Five participants flip the coin and obtain 28 $\mathrm{H}$ out of the total 50 throws

In [10]:
def MCMC(n_samples, accepted, rejected):
    a, b = 10, 5
    H, T = 28, 22

    #If chain already started, continue the chain
    if len(accepted):
        p_old=accepted[-1]

    # Otherwise, sample intial number from prior
    else:
        p_old = beta.rvs(a=10, b=5, size=1)

    for _ in range(n_samples):
        # Sample new number from Gaussian distribution centered around p_old
        p_new = multivariate_normal.rvs(mean=p_old, cov=0.01)

        # Our likelihood and prior functions
        likelihood = lambda p: binom.pmf(H, H + T, p).reshape(-1,)
        prior = lambda p: beta.pdf(p, a, b)
        # Acceptance
        alpha = likelihood(p_new) * prior(p_new) / (likelihood(p_old) * prior(p_old))

        u = np.random.random(1)
        if alpha > u :
            accepted.append(p_new)
            p_old = p_new
        else:
            rejected.append(p_new)

    return accepted, rejected

In [11]:
from IPython.display import display
import functools

fig2, ax2=plt.subplots(1,1)
accepted=[]
rejected=[]

x=np.linspace(0,1,100)
y_analytic = posterior(28,22,10,5,x)
burn_in=100
dropdown=widgets.Dropdown(
    options=[(1000, 1000), (2000, 2000), (5000, 5000)],
    value=1000,
    description='Number of samples: ',
    style={"description_width": "initial"}
)

button = widgets.Button(description="Draw samples")
output = widgets.Output()

display(dropdown,button, output)

def update(accepted, rejected):
    with output:
        n_samples=dropdown.value
        accepted, rejected = MCMC(n_samples, accepted, rejected)
        ax2.cla()
        ax2.hist(accepted[burn_in:], bins=50,density=True, histtype='step', color='blue', label='Drawn samples');
        ax2.plot(x,y_analytic, label='Analytical posterior', color='violet')
        ax2.fill_between(x, x*0, y_analytic, color="violet",alpha=0.2)
        ax2.legend()
        ax2.set_xlabel('$p$')
        ax2.set_ylabel('PDF')
        
def on_button_clicked(b, accepted, rejected):
    update(accepted, rejected)


button.on_click(functools.partial(on_button_clicked, accepted=accepted, rejected=rejected))




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Dropdown(description='Number of samples: ', options=((1000, 1000), (2000, 2000), (5000, 5000)), style=Descript…

Button(description='Draw samples', style=ButtonStyle())

Output()

## The case of the laser model

In the case of a time series, where we suppose $$y=\theta\cdot\mathbf{x} + \epsilon$$ with $$\epsilon \sim \mathcal{N}(0,\sigma^{2})$$
the likelihood function is given by $$\frac{\exp\left(-\frac 1 2 ({\mathbf y^*}-{\mathbf y})^\mathrm{T}{\boldsymbol\Sigma}^{-1}({\mathbf y^*}-{\mathbf y})\right)}{\sqrt{(2\pi)^k |\boldsymbol\Sigma|}} $$ with $y^*$ the predicted data points.

As seen previously, the MCMC algorithm evaluates the likelihood function for every new sample we draw. That means, we would have to compute a new FEM simulation for every new set of parameters, which is not feasable due to time constrains.

A way to avoid computing the FEM simulation everytime is to use surrogate models. A class of surrogate model is Gaussian Process Regression, which will be presented in the following.

# Gaussian Process Regression

The aim of the Gaussian Regression is to model the function as a multivariate Gaussian. 
A Gaussian process is a stochastic process with Gaussian distributions $$(y(x_1), y(x_2), ..., y(x_n)) \sim \mathcal{N}_n(\boldsymbol{\mu},\Sigma)$$

To specify a Gaussian distribution, we only need mean and variance $$X \sim \mathcal{N}(\mu, \Sigma)$$

To specify a Gaussian _process_, we need a mean and covariance _function_ $$y(\cdot) \sim GP(m(\cdot), k(\cdot,\cdot))$$

We are free to choose mean and covariance function, however they should be subject to some rules

Popular choices for mean are :
* m(x)=0
* m(x)=const
* m(x)=$\beta^T\mathbf{x}$

The kernel, which is a function of the location $$k(x,x')=\mathbb{C}ov(y(x),y(x'))$$
must be positive semi-definite to lead to a valid covariance matrix.

The choice of the kernel will dictate the space of functions that can represent our GP. A widely used kernel is the RBF kernel given by $$k(x,x')=\sigma^2\exp(-\frac{\|x-x'\|^2}{2l^2})$$

In the kernel you will notice that we have two parameters $\sigma$ and $l$. They are the variance and lengthscale of the kernel respectively. Those are two hyperparameters which we will need to tune in order to fit best the data. 

In a Gaussian regression, we start by first assuming the multivariate Gaussian has mean vector of zero and variance one, that is $$(y(x_1), y(x_2), ..., y(x_n)) \sim \mathcal{N}_n(\boldsymbol{0},I)$$

The multivariante gaussian distribution is then updated _conditional_ on the observed data point. That is, we update the mean vector and covariance matrix of the multivariate Gaussian with the newly acquired information.

For more in-depth details in how the conditional update is applied and the origin of the kernel, I strongly recommend going to [Gaussian Process Summer School](http://gpss.cc/gpss21/program)

While this all seems pretty abstract for now, let us explore the effect of using different kernel and their respective hyperparameters.


In [12]:
%%html
<iframe src="http://www.infinitecuriosity.org/vizgp/" width="1200" height="1000"></iframe>

Now that the fun is over, I would like to add more to what can be done with kernels.

A property of those kernels is that they can be added or multiplied together and still lead to positive semi-definite covoriance matrices. A discussion about kernel choice is given in [Kernel cookbook](https://www.cs.toronto.edu/~duvenaud/cookbook/)

Kernel are meant to pass _exactly_ on observed data point. However, if we know the variance of the observed data point, we can incorporate it in the model by adding a WhiteKernel to the kernel choice, which will simply add a variance term in the diagonal of the covariance matrix.

Until now, we have had function from $\mathbb{R}\rightarrow\mathbb{R}$. But we can also apply the same procedure for function $\mathbb{R}^n\rightarrow\mathbb{R}^n$ or $\mathbb{R}^n\rightarrow\mathbb{R}^n$. In that case, we can choose to either have the same kernel for all dimensions, or select different kernel for different dimensions.

# GPy for Gaussian Process Regression

Let us now see how all of this works in Python

We will start by import the relevant libraries and creating a dummy data set

In [13]:
import numpy as np
from matplotlib import pyplot as plt

#%matplotlib inline
import warnings

warnings.filterwarnings("ignore")

# GPy: Gaussian processes library
import GPy

We will now use our Gaussian process prior with some observed data to form a GP regression model. 

Suppose we have a data model for which we only have noisy observations, $y = f(x) + \epsilon$ at some small number of sample locations, $\mathbf{X}$. Here, we set up an example function

$$
    f(x) = -\cos(2\pi x) + \frac{1}{2}\sin(6\pi x)
$$
$$
    \mathbf{y} = f(\mathbf{X}) + \epsilon, \quad \epsilon \sim \mathcal{N}(0, 0.01)
$$

In [14]:
# lambda function, call f(x) to generate data
f = lambda x: -np.cos(2 * np.pi * x) + 0.5 * np.sin(6 * np.pi * x)

# 10 equally spaced sample locations
X = np.linspace(0.05, 0.95, 10)[:, None]

# y = f(X) + epsilon
Y = f(X) + np.random.normal(
    0.0, 0.1, (10, 1)
)  # note that np.random.normal takes mean and s.d. (not variance), 0.1^2 = 0.01

# Plot observations
fig3, ax3=plt.subplots(1,1)
ax3.plot(X, Y, "kx", mew=2)

# Annotate plot
ax3.set_xlabel("x"), plt.ylabel("f")
ax3.legend(labels=["sample points"]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can now fit a GP regression to the data using GPy. For that we will use a RBF kernel and optimize the parameters

In [15]:
# Setting up the kernel
k = GPy.kern.RBF(1, variance=1.0, lengthscale=0.1, name="rbf")
# Create the GP regression model
m = GPy.models.GPRegression(X, Y, k)
# Optimize the kernel hyperparameters, the lengthscale and the variance
m.optimize_restarts(3)
# Look at the model parameters
m

Optimization restart 1/3, f = 9.55749646765872
Optimization restart 2/3, f = 9.557495378215801
Optimization restart 3/3, f = 9.557495528670088


GP_regression.,value,constraints,priors
rbf.variance,0.7124504743054618,+ve,
rbf.lengthscale,0.10034634983104936,+ve,
Gaussian_noise.variance,4.717638732837407e-09,+ve,


In [16]:
def plot_gp(X, m, C, training_points=None):
    """ Plotting utility to plot a GP fit with 95% confidence interval """
    # Plot 95% confidence interval
    plt.fill_between(
        X[:, 0],
        m[:, 0] - 1.96 * np.sqrt(np.diag(C)),
        m[:, 0] + 1.96 * np.sqrt(np.diag(C)),
        alpha=0.5,
    )
    # Plot GP mean and initial training points
    plt.plot(X, m, "-")
    plt.legend(labels=["GP fit"])

    plt.xlabel("x"), plt.ylabel("f")

    # Plot training points if included
    if training_points is not None:
        X_, Y_ = training_points
        plt.plot(X_, Y_, "kx", mew=2)
        plt.legend(labels=["GP fit", "sample points"])

In [17]:
Xnew = np.linspace(-0.05, 1.05, 100)[:, None]
# Predict new data points
# Get mean and covariance of optimised GP
mean, Cov = m.predict_noiseless(Xnew, full_cov=True)

# Plot the GP fit mean and covariance
plt.figure(figsize=(10, 5))
plot_gp(Xnew, mean, Cov, training_points=(X, Y))
plt.plot(Xnew, f(Xnew), "r:", lw=3);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

For more information on how to use GPy, please visit [GPSS Lab](http://gpss.cc/gpss21/labs) for a general introduction, [GP Examples](https://nbviewer.org/github/SheffieldML/notebook/blob/master/GPy/index.ipynb) for specific examples on how to use GPy and the documentation can be found here : [GPy Docs](https://gpy.readthedocs.io/en/deploy/).

While GPy is stable, it is no longer under active developement. Alternatives are [GPytorch](https://gpytorch.ai/) and [GPyFlow](https://gpflow.readthedocs.io/en/master/intro.html) based on Pytorch and TensorFlow respectively, to leverage GPU speed up and training on big data sets. It should be noted that a less extensive GP Regression is provided by [Scikit-learn](https://scikit-learn.org/stable/modules/gaussian_process.html). Another library is [PyMC3](https://docs.pymc.io/en/stable/) which is specialized in probabilistic learning. I find the latter to be the least intuitive of them all to set up a model.